### 生成式半监督模型
- 假设训练数据集$D$中的样本都是由同一个潜在的模型生成的。

缺点：假设的生成式模型必须与真实数据分布吻合。如果不吻合则可能效果很差。

#### 生成式高斯混合模型

### 图半监督学习
用图$G(V,E)$来表示一个样本集$D$：
> 1. 顶点$V$：代表每个样本$\vec{x}_i$
> 2. 边$E$：代表样本之间的相似度。若两个样本相似度高，则对应的顶点之间存在一条边。边的权重正比于样本之间的相似度。

#### 标记传播算法（Label Propagation）
缺点：存储开销大，难以直接处理大规模数据；而且对于新的样本加入，需要对原图重构并进行标记传播。

基本思想：将一个节点的邻居节点的标签中数量最多的标签作为该节点自身的标签。给每个节点添加标签以代表它所属的社区，并通过标签的传播形成同一标签的社区结构。

传播过程：
> 1. 初始时，给每个节点一个唯一的标签；
> 2. 每个节点使用其邻居节点的标签中最多的标签来更新自身的标签；
> 3. 反复执行步骤2，直到每个节点的标签都不再发生变化为止。

边集$E$用矩阵表示为：$$(W)_{i,j}=\left\{\begin{matrix}
exp(-\frac{||\vec{x}_i-\vec{x}_j||_2^2}{2\sigma ^2}),\ \ if i\neq j\\ 
0,\ \ \ otherwise
\end{matrix}\right.$$
$$i,j\in \{1,2,\cdots,N\}$$
其中，$\sigma$为超参数。

### 实战

In [13]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn import datasets
from sklearn.semi_supervised import LabelPropagation, LabelSpreading

In [2]:
def load_data():
    digits = datasets.load_digits()
    rng = np.random.RandomState(0)
    indices = np.arange(len(digits.data))
    rng.shuffle(indices)
    X = digits.data[indices]
    y = digits.target[indices]
    n_labeled_points = int(len(y)/10)  # 只有10%的样本有标记
    unlabeled_indices = np.arange(len(y))[n_labeled_points:]  # 后面90%的样本未标记
    return X, y, unlabeled_indices

#### LabelPropagation
**sklearn.semi_supervised.LabelPropagation(kernel='rbf', gamma=20, n_neighbors=7, alpha=None, max_iter=1000, tol=0.001, n_jobs=None)**
> 1. kernel：字符串，指定核函数
    - rbf：距离为$exp(-\gamma |x-y|^2),\ \gamma > 0$
    - knn：如果$x$是$y$的$k$近邻，则距离为1,；否则距离为0。
    - rbf核计算量较大，且距离矩阵是对称的；knn核计算量较小，距离矩阵是稀疏矩阵，且距离矩阵是不对称的。
> 2. gamma：浮点数，rbf核的参数
> 3. n_neighbors：整数，knn核的参数
> 4. alpha：浮点数，为折中系数$\alpha$

属性：
> 1. X_：输入数组
> 2. classes_：分类问题中，类别标记数组
> 3. label_distributions_：数组，给出了每个样本的标记分布
> 4. transduction_：给出每个样本计算出的标记
> 5. n_iter_：给出迭代次数

方法：
> 1. fit(X, y)：训练模型
> 2. predict(X)：预测标记
> 3. predict_proba(X)：预测对于每个标记出现的概率
> 4. score(X, y[, sample_weight])：评估在测试集上的预测准确率

In [15]:
def test_LabelPropagation(*data):
    X, y, unlabeled_indices = data
    y_train = np.copy(y)
    y_train[unlabeled_indices] = -1
    clf = LabelPropagation(max_iter=100, kernel='rbf', gamma=0.1)
    clf.fit(X, y_train)
    true_labels = y[unlabeled_indices]  # 真实标记
    print("Accuracy: %f"% clf.score(X[unlabeled_indices], true_labels))
    
data = load_data()
test_LabelPropagation(*data)

Accuracy: 0.948084


#### LabelSpreading
**sklearn.semi_supervised.LabelSpreading(kernel='rbf', gamma=20, n_neighbors=7, alpha=0.2, max_iter=30, tol=0.001, n_jobs=None)**
